In [2]:
import math

def calculate_uncertainty():
    print("="*40)
    print("  RLC串联谐振实验 - 不确定度计算器")
    print("="*40)
    print("注意：请输入纯数字，不要带单位。")
    print("-" * 40)

    # --- 第一步：输入仪器参数（只需输入一次） ---
    print("\n【1. 仪器参数设置】")
    try:
        # 电压表参数
        print(">> 请查看电压表背面或说明书 (格式如: ±0.5% reading + 2 digits)")
        v_acc_percent = float(input("   电压表精度百分比 a (例如0.5，输入0.5): ")) / 100
        v_acc_digits = int(input("   电压表尾数跳动字数 n (例如2，输入2): "))
        
        # 频率参数
        # RLC实验中，频率的主要误差来源不是仪器精度，而是寻找半功率点时的“宽容度”
        f_jitter = float(input("   频率判定的估读误差 (例如你觉得±5Hz内电压没变，就输5): "))
    except ValueError:
        print("输入错误，请输入数字。")
        return

    # --- 第二步：输入测量数据 ---
    print("\n【2. 输入实验测量数据】")
    try:
        # 1. 谐振点
        f0 = float(input("   测得的谐振频率 f0 (Hz): "))
        u_max = float(input("   谐振时的电压 U_max (V): "))
        u_res_max = float(input(f"   读取 U_max 时电压表的分辨率 (例如 0.01 或 0.001): "))

        # 2. 半功率点 1
        f1 = float(input("   半功率点频率 f1 (Hz): "))
        # 这里的电压通常不需要测很不确定度，因为它是作为目标值去找频率的，但为了完整性可以算
        
        # 3. 半功率点 2
        f2 = float(input("   半功率点频率 f2 (Hz): "))
        
    except ValueError:
        print("输入数据有误，请重试。")
        return

    # --- 第三步：计算电压不确定度 u(U) ---
    # 公式：u_B = (a% * Reading + n * Resolution) / sqrt(3)
    # 假设均匀分布 (Rectangular Distribution)
    
    def calc_u_voltage(reading, resolution):
        limit_error = (reading * v_acc_percent) + (v_acc_digits * resolution)
        u_b = limit_error / math.sqrt(3)
        return u_b

    u_u_max = calc_u_voltage(u_max, u_res_max)

    # --- 第四步：计算频率不确定度 u(f) ---
    # 这里主要考虑估读误差，假设为矩形分布或正态分布，通常实验取 u = jitter
    # 如果保守一点，可以除以根号3，这里为了保险直接用输入的 jitter 作为标准不确定度
    u_f = f_jitter 

    # --- 第五步：计算 Q 值及其不确定度 u(Q) ---
    # Q = f0 / (f2 - f1)
    bandwidth = f2 - f1
    q_val = f0 / bandwidth

    # 合成不确定度
    # 1. 带宽的不确定度 u(BW) = sqrt(u(f1)^2 + u(f2)^2)
    u_bandwidth = math.sqrt(u_f**2 + u_f**2)

    # 2. Q的不确定度传递公式: u(Q)/Q = sqrt( (u(f0)/f0)^2 + (u(BW)/BW)^2 )
    relative_u_f0 = (u_f / f0) ** 2
    relative_u_bw = (u_bandwidth / bandwidth) ** 2
    
    u_q = q_val * math.sqrt(relative_u_f0 + relative_u_bw)

    # --- 第六步：输出结果 ---
    print("\n" + "="*40)
    print("           计算结果报告")
    print("="*40)
    
    print(f"【电压 U_max】")
    print(f"  测量值: {u_max:.4f} V")
    print(f"  不确定度 u(U): {u_u_max:.4f} V")
    print(f"  结果表示: ({u_max:.4f} ± {u_u_max:.4f}) V")
    
    print(f"\n【频率 f】")
    print(f"  f0: {f0} Hz, f1: {f1} Hz, f2: {f2} Hz")
    print(f"  设定的频率不确定度 u(f): {u_f} Hz")
    
    print(f"\n【品质因数 Q】")
    print(f"  带宽 (f2 - f1): {bandwidth:.2f} Hz")
    print(f"  计算得到的 Q 值: {q_val:.4f}")
    print(f"  Q 的标准不确定度 u(Q): {u_q:.4f}")
    print(f"  相对不确定度: {(u_q/q_val)*100:.2f}%")
    print("-" * 40)
    print(f"最终结果建议: Q = {q_val:.2f} ± {u_q:.2f}")
    print("="*40)

if __name__ == "__main__":
    calculate_uncertainty()

  RLC串联谐振实验 - 不确定度计算器
注意：请输入纯数字，不要带单位。
----------------------------------------

【1. 仪器参数设置】
>> 请查看电压表背面或说明书 (格式如: ±0.5% reading + 2 digits)

【2. 输入实验测量数据】

           计算结果报告
【电压 U_max】
  测量值: 0.4730 V
  不确定度 u(U): 0.0025 V
  结果表示: (0.4730 ± 0.0025) V

【频率 f】
  f0: 15751.0 Hz, f1: 14981.0 Hz, f2: 16561.0 Hz
  设定的频率不确定度 u(f): 8.0 Hz

【品质因数 Q】
  带宽 (f2 - f1): 1580.00 Hz
  计算得到的 Q 值: 9.9690
  Q 的标准不确定度 u(Q): 0.0716
  相对不确定度: 0.72%
----------------------------------------
最终结果建议: Q = 9.97 ± 0.07
